# Banana Run

This is my solution to the bananas problem for the Udacity [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893). Udacity adapted this problem and runtime environment from the Unity's ML-Agents [Banana Collector environment](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#banana-collector). This class runs from May 1, 2019, to August 24, 2019, and this is the first of 5 projects, due June 18th. I started this on May 14th.

The purpose of this Notebook is educational; to reinforce and deepen ;) my understanding of value-based methods for Deep Reinforcement Learning (DRL). I will attempt to demonstrate a useful skill, in my ability to quickly construct a good solution to a problem which is a stand-in for a large class of real-world problems. I will also attempt to develop an academic comprehension of DNN value-based methods for reinforcement learning by analysing their performance characteristics and comparing them to other forms of machine learning. It is that kind of understanding that may lead me to invent and evaluate novel solutions to similar problems.

## The Problem

![Bananas](banana.gif "Bananas!")

The problem domain is a virtual world built in the Unity envionment in which yellow and blue bananas fall (apparently at random intervals) from the sky. The agent's objective is to collect as many yellow bananas and as few blue bananas as posible. More precisely, the agent is rewarded +1 point for collecting a yellow banana and rewarded -1 point for collecting blue bananas, and the objective is to get as many points as possible during a fixed-time episode. The agent is considered successful if it is able to average a reward of at least +13 over the course of 100 consecutive episode.

Our agent's inputs are 37 dimentional, and include the agent's velocity, along with ray-based perception of objects around the agent's forward direction. Given this information, the agent has to learn how to best select actions. Four discrete actions are available, corresponding to:

1. move forward.
2. move backward.
3. turn left.
4. turn right.

## Training

In [10]:
%load_ext autoreload
%autoreload 1

from unityagents import UnityEnvironment
import numpy as np
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
env = UnityEnvironment(file_name="../Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


In [5]:
%aimport dqn_agent
%aimport model
from collections import namedtuple

Device = cuda:0


In [6]:
episode_datum = namedtuple('episode_datum', field_names=['score'])
log = []

In [7]:
def running_mean(x, N=100):
    return x[np.convolve([l.score for l in log], np.ones((10,))/10., mode="valid")

SyntaxError: unexpected EOF while parsing (<ipython-input-7-4387ee7168cb>, line 2)

In [8]:
agent = dqn_agent.Agent(state_size, action_size, 0)

In [9]:
episodes = 2500
total_score = 0
eps = 1.0
for e in range(episodes):
    # reset the environment
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations[0]

    eps = max(0.001, eps*.995)
    score = 0.
    done = False
    while not done:
        action = agent.act(state, eps)                      # select an action
        env_info = env.step(action)[brain_name]             # send the action to the environment
        next_state = env_info.vector_observations[0]        # get the next state
        reward = env_info.rewards[0]                        # get the reward
        done = env_info.local_done[0]                       # see if episode has finished
        agent.step(state, action, reward, next_state, done) # maybe learn something
        score += reward                                     # update the score
        state = next_state                                  # roll over the state to next time step
    
    log.append(episode_datum(score))
    windowed_mean = sum([l.score for l in log[-100:]])/100.0
    print('{:d}: score: {:2.0f} running-mean: {:4.2f}'.format(e, score, windowed_mean), end="\r")
    if windowed_mean >= 13:
        break

print()

2499: score: 13.000000 running-mean: 10.47


In [11]:
pickle.dump(log, open('dqn1.p', 'wb'))

In [ ]:
env.close()

## Evaluation

## Conclusions

## Future Work

## References